## Tutorial for Real-Time fMRI Cloud-Based Framework

### Brief Overview
This tutorial walks through an example using our cloud-based software framework for real-time fMRI studies, henceforth **rtcloud framework**. We will begin by starting the projectInterface web server, which will wrap our sample.py script. We will then use a brainiak function to create synthetic DICOM data to simulate data produced from the scanner at 2 TRs. Finally, we will open the web server on a localhost that can be displayed in a cell (although you can also open it on a different tab in your browser).

## Start the projectInterface Web Server

In [ ]:
### ADD FULL PATH TO RTCLOUD REPO AND NOTEBOOK ###
import os
path_to_rtcloud = os.getenv('RTCLOUD_PATH')
if path_to_rtcloud == None:
    print("Please set RTCLOUD_PATH, see instructions")
    raise ValueError
    
path_to_notebook = os.getcwd()  # check and change notebook path as needed

# the python path to the rtcloud repo is set here (to access rtCommon functions)
import sys
sys.path.append(path_to_rtcloud)

# load important modules
from rtCommon.projectInterface import Web
from rtCommon.structDict import StructDict

# declare the scripts that will be wrapped by the projectInterface and accessed through the webServer
scriptToRun = os.path.join(path_to_notebook, 'sample.py')
initScript = os.path.join(path_to_notebook, 'initialize.py')
finalizeScript = os.path.join(path_to_notebook, 'finalize.py')

# set up the config and parameters for this rtcloud tutorial
# NOTE: you can also change these parameters in the Settings tab on the web server
config = StructDict({
    'title' : 'rtCloud Tutorial',
    'sessionId' : '20200101T120000',
    'subjectName' : '001_synthetic',
    'datestr' : '20200101',
    
    'isSynthetic' : True,
    'numSynthetic' : 20,
    'imgDir' : '/tmp/notebook-simdata',
    
    'subjectNum' : 101,
    'subjectDay' : 1,
    'sessionNum' : 1,
    'runNum' : [1],
    'scanNum' : [13],

    # Plotting settings
    'plotTitle' : 'Realtime Plot',
    'plotXLabel' : 'Sample #',
    'plotYLabel' : 'Value',
    'plotXRangeLow' : 0,
    'plotXRangeHigh' : 20,
    'plotYRangeLow' : -1,
    'plotYRangeHigh' : 1,
    'plotAutoRangeX' : False,
    'plotAutoRangeY' : True,

    # important variables about the dicom files
    'dicomNamePattern' : "rt_{run:03d}.dcm",
    'minExpectedDicomSize' : 200000,
})

params = StructDict({
    'fmriPyScript': scriptToRun,
    'initScript': initScript,
    'finalizeScript': finalizeScript,
    'filesremote': False,
    'port': 8889
})

# start the web server
web = Web()
try:
    web.start(params, config, testMode=True)
    print(true)
except RuntimeError as err:
    # ignore event loop already running error
    if str(err) != 'This event loop is already running':
        raise

#### PLEASE NOTE: If you want to re-start the projectInterface web server, you will first have to SHUTDOWN the kernel.

## Start the Synthetic Data Generator

We will be using a brainiak function to create synthetic fMRI data. After you run the following cell, you can execute `sim_settings` in a separate cell to take a look at the different parameter settings. For instance, you will find that the synthetic data is being produced at a rate of 2 TRs.

In [ ]:
import threading
import brainiak.utils.fmrisim_real_time_generator as sim

sim_settings = sim.default_settings
sim_settings['save_dicom'] = True
sim_settings['save_realtime'] = True
sim_settings['numTRs'] = 20

outdir = '/tmp/notebook-simdata'
if not os.path.exists(outdir):
    os.makedirs(outdir)

# run sim.generate_data(outdir, sim_settings) as a thread
syndata_thread = threading.Thread(name='syndata', 
                                  target=sim.generate_data, 
                                  args=(outdir, sim_settings))
syndata_thread.setDaemon(True)
syndata_thread.start()

#### PLEASE NOTE: If you want to re-run the synthetic data generator in "real-time", you have to delete the `/tmp/notebook-simdata` folder.

### Open the web server on the localhost (you can also this on a separate browser tab).

In [ ]:
%%html
<iframe src="http://localhost:8889" width="800" height="600"></iframe>